In [34]:
from libpysal.io import 

In [35]:
io.

In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import numpy as np
import math
from scipy.optimize import minimize, Bounds
from scipy.stats import norm, ks_2samp
from haversine import haversine
from scipy.spatial.distance import euclidean
from shapely.geometry import Polygon, MultiPolygon
from matplotlib import pyplot as plt, ticker

## Obtenção e preparo da amostra

In [ ]:
entregas = pd.read_csv('Dados/New_query_2023_07_15.csv', parse_dates=['event_day', 'event_timestamp', 'event_timestamp_hour']).sort_values('event_timestamp')

In [ ]:
entregas_por_rest = entregas.groupby('frn_id')['order_id'].nunique().reset_index()

In [ ]:
tamanho_amostra = len(entregas_por_rest)

In [ ]:
rests_amostra = entregas_por_rest[entregas_por_rest.order_id > 0][['frn_id']].drop_duplicates().sample(n=tamanho_amostra, random_state=123)

In [ ]:
entregas_amostra = entregas.merge(rests_amostra, on='frn_id')

In [ ]:
inicio_fim_rotas = entregas_amostra.groupby('route_id', as_index=False).agg({'event_timestamp': ['min', 'max']}).droplevel(0, 1).rename({'': 'route_id', 'min': 'inicio_rota', 'max': 'fim_rota'}, axis='columns')

In [ ]:
entregas_amostra = entregas_amostra.merge(inicio_fim_rotas, how='inner', on='route_id')

## Funções Processo Espacial de Nascimento e Morte

In [23]:
def montar_janela(t0, m, delta_t):
    """
    Obtém uma janela de m datetimes iniciando em t0 e saltando a cada delta_t
    
    :param t0: momento inicial da janela
    :param m: número de momentos que compõem a janela
    :param delta_t: tamanho do salto de tempo entre cada momento t
    :returns: uma lista de momentos compondo uma janela
    """
    return [t0 + dt.timedelta(days=m) for m in range(0, m * delta_t, delta_t)]

In [3]:
def configurar_janela(df, evt, fim, obj, lng, lat, t0, m, delta_t):
    """
    Obtém a configuração de estado de cada momento de uma janela
    
    :param df: dataframe com os dados para popular a janela
    :param evt: coluna com as atualizações de cada evento
    :param fim: coluna com o término de cada evento
    :param obj: coluna com o objeto que nasce/morre/move
    :param lat: coluna com as latitudes do evento
    :param lng: coluna com as longitudes do evento
    :param t0: momento inicial da janela
    :param m: número de momentos que compõem a janela
    :param delta_t: tamanho do salto de tempo entre cada momento t
    :returns: uma janela com suas configurações em cada momento t
    """
    j = montar_janela(t0, m, delta_t)
    X = []

    for t in j:
        objs_vivos = df[(df[evt] <= t) & (df[fim] > t)]
        ids = objs_vivos[obj].unique()
        coords = objs_vivos.drop_duplicates(subset=[obj], keep='last')[[lng, lat]].to_numpy()
        X.append({'t': t, 'n': len(coords), 'id': ids, 'xy': coords})
        
    return X

In [4]:
def hausdorff(janela, chave_t, chave_xy, dist_fn):
    matriz_hausdorff = []

    for i, Ji in enumerate(janela):
        linha = []
        for j, Jj in enumerate(janela):
            if i < j:
                matriz_dist = np.array([np.array([dist_fn(k, l) for l in Jj[chave_xy]]) for k in Ji[chave_xy]])
                dist = np.max(np.concatenate((np.amin(matriz_dist, axis=1), np.amin(np.transpose(matriz_dist), axis=1))))
            elif i > j:
                dist = matriz_hausdorff[j][i]
            else:
                dist = 0

            linha.append(dist)
        matriz_hausdorff.append(np.array(linha))
    matriz_hausdorff = np.array(matriz_hausdorff)

    return matriz_hausdorff

In [27]:
def gerar_alfas(janela):
    beta = [len(np.setdiff1d(j['id'], i['id'])) for (i, j) in zip(janela[:-1] , janela[1:])]
    delta = [len(np.setdiff1d(i['id'], j['id'])) for (i, j) in zip(janela[:-1] , janela[1:])]
    
    return {'m': range(len(janela)), 'beta': beta, 'delta': delta}

In [28]:
janela = configurar_janela(lojas, 'activation_date', 'status_modified_at', 'frn_id', 'longitude', 'latitude', dt.datetime(2022, 1, 1), 355, delta_t)
janela

[{'t': datetime.datetime(2022, 1, 1, 0, 0),
  'n': 9230,
  'id': array([1049737, 1048646, 1045466, ...,   61386,   61247,   61772],
        dtype=int64),
  'xy': array([[-38.504908,  -3.750204],
         [-38.552888,  -3.738778],
         [-38.574989,  -3.775847],
         ...,
         [-38.513706,  -3.742418],
         [-38.538991,  -3.807521],
         [-38.503288,  -3.72487 ]])},
 {'t': datetime.datetime(2022, 1, 2, 0, 0),
  'n': 9229,
  'id': array([1049737, 1048646, 1045466, ...,   61386,   61247,   61772],
        dtype=int64),
  'xy': array([[-38.504908,  -3.750204],
         [-38.552888,  -3.738778],
         [-38.574989,  -3.775847],
         ...,
         [-38.513706,  -3.742418],
         [-38.538991,  -3.807521],
         [-38.503288,  -3.72487 ]])},
 {'t': datetime.datetime(2022, 1, 3, 0, 0),
  'n': 9234,
  'id': array([1049737, 1048646, 1045466, ...,   61386,   61247,   61772],
        dtype=int64),
  'xy': array([[-38.504908,  -3.750204],
         [-38.552888,  -3.73877

In [5]:
def estimador_alfa(delta_n, delta_t, h, matriz_dist,K=norm.pdf):
    Kt = K(matriz_dist, 0, h).flatten()
    n = len(Kt) - 1
    Kt = np.delete(Kt, [n])
    t_chapeu = sum(Kt * delta_t)

    return np.sum(np.divide(np.resize(delta_n, n) * Kt, t_chapeu, where=t_chapeu != 0))

In [6]:
def log_verossimilhanca(h, janela, delta_n, delta_t, matriz_dist, K=norm.pdf):
    n = len(janela) - 1
    alfa_i = []
    
    for i in range(n):
        saltos_t = np.delete(delta_n, [i])
        dists_t = np.delete(matriz_dist[i], [i])
        alfa_i.append(estimador_alfa(saltos_t, delta_t, h, dists_t, K=K))
        
    alfa_i = np.array(alfa_i)
    return -np.sum(delta_n * np.log(alfa_i)) + np.sum(delta_t * alfa_i)

In [7]:
def maximizar_verossimilhanca(janela, delta_n, delta_t, matriz_dist, K=norm.pdf):
    iniciar = np.quantile(matriz_dist, 0.1) / 2
    maximo = np.max(matriz_dist)
    
    return minimize(fun=log_verossimilhanca, x0=(iniciar,), args=(janela, delta_n, delta_t, matriz_dist, K), bounds=Bounds(0.00001, maximo)).x

## Gerador dos estimadores

In [ ]:
jan_teste = [
    {'t': dt.datetime(2023, 1, 10, 0, 15), 'xy': np.array([[29.30464, 129.0295], [175.52776, 85.88214]])},
    {'t': dt.datetime(2023, 1, 10, 0, 30), 'xy': np.array([[29.1836, 129.0577], [175.7906, 86.25676], [107.1100, 176.47044]])}
]

hausdorff(jan_teste, 't', 'xy', euclidean)

In [8]:
def gerar_estimadores(df, evt, fim, obj, lat, lng, ini_janela, m, delta_t):
    janela = configurar_janela(df, evt, fim, obj, lat, lng, ini_janela, m, delta_t)
    matriz_dists = hausdorff(janela, 't', 'xy', euclidean)
    beta = [len(np.setdiff1d(j['id'], i['id'])) for (i, j) in zip(janela[:-1] , janela[1:])]
    delta = [len(np.setdiff1d(i['id'], j['id'])) for (i, j) in zip(janela[:-1] , janela[1:])]
    h_chapeu_beta = maximizar_verossimilhanca(janela, beta, delta_t, matriz_dists)
    beta_chapeu = [estimador_alfa(beta, delta_t, h_chapeu_beta, l) for l in matriz_dists]
    h_chapeu_delta = maximizar_verossimilhanca(janela, delta, delta_t, matriz_dists)
    delta_chapeu = [estimador_alfa(delta, delta_t, h_chapeu_delta, l) for l in matriz_dists]
    
    return {'h_chapeu_b': h_chapeu_beta,
            'b': beta,
            'b_chapeu': beta_chapeu,
            'h_chapeu_d': h_chapeu_delta,
            'd': delta,
            'd_chapeu': delta_chapeu}

## Gradeamento da área da cidade

In [ ]:
fortaleza = gpd.read_file('Dados/Densidade_Populacional_por_Bairros.zip')

In [ ]:
def arred_baixo(num, dec):
    return math.floor(num * 10 ** dec) / 10 ** dec

In [ ]:
def criar_celula(lat, lng, passo):
    return Polygon([(lat, lng), (lat + passo, lng), (lat + passo, lng + passo), (lat, lng + passo)])

In [ ]:
x_min, y_min, x_max, y_max = (arred_baixo(b, 2) for b in fortaleza.geometry.unary_union.bounds)

In [ ]:
p = 0.01

In [ ]:
grade_completa = MultiPolygon((criar_celula(x, y, p) for x in np.arange(x_min - p, x_max + p, p) for y in np.arange(y_min - p, y_max + p, p)))   

In [ ]:
grade_fortaleza = MultiPolygon([c for c in grade_completa.geoms if c.intersects(fortaleza.unary_union)])

In [ ]:
df_grade_fortaleza = gpd.GeoDataFrame({}, geometry=gpd.GeoSeries(grade_fortaleza), crs='EPSG:4326').explode()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

fortaleza.plot(ax=ax1)
ax2.set_xlim(ax1.get_xlim())
ax2.set_ylim(ax1.get_ylim())
df_grade_fortaleza.boundary.plot(ax=ax2)

## Estimativas gerais de ^b e ^d

In [ ]:
delta_t = 5

In [ ]:
estimadores = gerar_estimadores(entregas_amostra, 'event_timestamp', 'fim_rota', 'driver_uuid', 'drv_lng', 'drv_lat', dt.datetime(2023, 1, 10, 0, 5, 0, 0), 286, delta_t)

In [ ]:
fig, (axn, axd) = plt.subplots(ncols=2, nrows=1, figsize=(16, 4))

plt1 = axn.plot(range(len(estimadores['b'])), estimadores['b'], color='b', label='emp')
axn_2 = axn.twiny()
plt2 = axn_2.plot(range(len(estimadores['b_chapeu'])), np.array(estimadores['b_chapeu']) * delta_t, '--k', label='est')
plts = plt1+plt2
labs = [p.get_label() for p in plts]
axn.legend(plts, labs, loc=0)

plt1 = axd.plot(range(len(estimadores['d'])), estimadores['d'], color='r', label='emp')
axd_2 = axd.twiny()
plt2 = axd_2.plot(range(len(estimadores['d_chapeu'])), np.array(estimadores['d_chapeu']) * delta_t, '--k', label='est')
plts = plt1+plt2
labs = [p.get_label() for p in plts]
axd.legend(plts, labs, loc=0)

In [ ]:
ks_2samp(estimadores['b'][:-1], estimadores['b_chapeu'])

In [ ]:
entregas = gpd.GeoDataFrame(entregas, geometry=gpd.points_from_xy(entregas.cli_lng, entregas.cli_lat), crs='EPSG:4326')

In [ ]:
entregas_grade = df_grade_fortaleza.sjoin(entregas, how='inner', predicate='contains')

In [16]:
lojas = pd.read_csv('Dados/lojas.csv', parse_dates=['status_modified_at', 'activation_date'])

In [18]:
lojas['status_modified_at'] = lojas.apply(lambda l: dt.date.today() if l.status_indication_description != 'Inativo (churn)'' else l.status_modified_at, axis=1)

In [22]:
delta_t = 1

In [21]:
lojas[lojas.status_indication_description != 'Inativo (churn)']

,frn_id,trading_name,dish_description,latitude,longitude,status_indication_description,status_modified_at,activation_date
11,1041047,Buba Burguer,Lanches,-3.820246,-38.588828,Ativo,2023-07-20,2020-05-03
12,1076388,Parada Fit,Comida Saudável,-3.722464,-38.568006,Ativo,2023-07-20,2020-05-15
16,1039818,Fabiana Castro,Pizza,-3.800541,-38.546185,Ativo,2023-07-20,2020-04-10
19,1074937,Corpus Suplementos,Comida Saudável,-3.732750,-38.560580,Ativo,2023-07-20,2020-05-04
32,1070296,Bar E Pizzaria O Murilo,Comida Brasileira,-3.798442,-38.548647,Ativo,2023-07-20,2020-04-28
...,...,...,...,...,...,...,...,...
14949,165856,Pizzaria Wifi,Pizza,-3.734820,-38.540198,Ativo,2023-07-20,2018-09-04
14951,160494,Tempero Da Stefany,Comida Brasileira,-3.726119,-38.461682,Ativo,2023-07-20,2018-08-08
14967,60072,Camarão Da Cidade,Frutos do Mar,-3.778476,-38.482895,Ativo,2023-07-20,2017-05-23
14971,60234,Kipuro Açai,Açaí,-3.710823,-38.593505,Ativo,2023-07-20,2017-06-15


In [26]:
estimadores = gerar_estimadores(lojas, 'activation_date', 'status_modified_at', 'frn_id', 'longitude', 'latitude', dt.datetime(2022, 1, 1), 355, delta_t)

KeyboardInterrupt: 